In [1]:
import zipfile
import os

# Paths to your zip files
zip_paths = [
    "/content/augmented_images.zip",
    "/content/augmented_images(white throated kingfisher).zip",
    "/content/augmented_images(red vented bulbul).zip"
]

# Unzipping function
def unzip_files(zip_paths, extract_to_base):
    for zip_path in zip_paths:
        # Get the folder name based on the zip file name
        folder_name = os.path.splitext(os.path.basename(zip_path))[0]
        extract_to = os.path.join(extract_to_base, folder_name)

        os.makedirs(extract_to, exist_ok=True)
        print(f"Unzipping {zip_path} to {extract_to}...")

        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Finished unzipping {zip_path} to {extract_to}")

# Unzip all files to separate directories for each bird
extracted_base_dir = "/content/extracted_images"
unzip_files(zip_paths, extracted_base_dir)


Unzipping /content/augmented_images.zip to /content/extracted_images/augmented_images...
Finished unzipping /content/augmented_images.zip to /content/extracted_images/augmented_images
Unzipping /content/augmented_images(white throated kingfisher).zip to /content/extracted_images/augmented_images(white throated kingfisher)...
Finished unzipping /content/augmented_images(white throated kingfisher).zip to /content/extracted_images/augmented_images(white throated kingfisher)
Unzipping /content/augmented_images(red vented bulbul).zip to /content/extracted_images/augmented_images(red vented bulbul)...
Finished unzipping /content/augmented_images(red vented bulbul).zip to /content/extracted_images/augmented_images(red vented bulbul)


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Function to split and organize datasets
def split_and_organize_datasets(base_extracted_dir, split_base_dir, test_size=0.2, val_size=0.2):
    # Clear or create split directories
    if os.path.exists(split_base_dir):
        shutil.rmtree(split_base_dir)
    os.makedirs(split_base_dir, exist_ok=True)

    # Create subdirectories for train, validation, and test
    for split in ['train', 'val', 'test']:
        os.makedirs(os.path.join(split_base_dir, split), exist_ok=True)

    # Loop through each bird folder and split its images
    for bird_folder in os.listdir(base_extracted_dir):
        bird_dir = os.path.join(base_extracted_dir, bird_folder)
        if not os.path.isdir(bird_dir):
            continue

        print(f"Processing {bird_folder}...")

        # Get all image files in the current bird folder
        images = os.listdir(bird_dir)
        images = [img for img in images if img.endswith(('.jpg', '.jpeg', '.png'))]  # Filter valid image files

        if len(images) == 0:
            print(f"No valid image files found in {bird_folder}")
            continue

        # Split the images into train, test, and validation sets
        train_imgs, test_imgs = train_test_split(images, test_size=test_size, random_state=42)
        train_imgs, val_imgs = train_test_split(train_imgs, test_size=val_size, random_state=42)

        # Copy images to the corresponding directories
        for split, imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
            split_dir = os.path.join(split_base_dir, split, bird_folder)
            os.makedirs(split_dir, exist_ok=True)
            for img in imgs:
                shutil.copy(os.path.join(bird_dir, img), os.path.join(split_dir, img))

# Path to your extracted dataset directory
base_extracted_dir = "/content/extracted_images"
split_base_dir = "/content/split_datasets"
split_and_organize_datasets(base_extracted_dir, split_base_dir)


Processing augmented_images...
Processing augmented_images(red vented bulbul)...
Processing augmented_images(white throated kingfisher)...


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Image size and batch size settings
img_size = (224, 224)
batch_size = 32

# Create the ImageDataGenerator objects for train, validation, and test sets
train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Define the directories for train, validation, and test
train_dir = os.path.join("/content/split_datasets", 'train')
val_dir = os.path.join("/content/split_datasets", 'val')
test_dir = os.path.join("/content/split_datasets", 'test')

# Create data generators for train, validation, and test
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 1920 images belonging to 3 classes.
Found 480 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load the ResNet50 model pre-trained on ImageNet, without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of ResNet50
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling to reduce the output size
x = Dense(128, activation='relu')(x)  # Fully connected layer with ReLU activation
predictions = Dense(train_gen.num_classes, activation='softmax')(x)  # Output layer with softmax for multi-class classification

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of ResNet50 (base model) to prevent training them
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train the model
epochs = 10
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    verbose=1
)


Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 427s 7s/step - accuracy: 0.3323 - loss: 1.1579 - val_accuracy: 0.4167 - val_loss: 1.0744
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 498s 8s/step - accuracy: 0.3981 - loss: 1.0767 - val_accuracy: 0.4437 - val_loss: 1.0502
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 411s 7s/step - accuracy: 0.4621 - loss: 1.0483 - val_accuracy: 0.4583 - val_loss: 1.0361
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 437s 7s/step - accuracy: 0.4790 - loss: 1.0317 - val_accuracy: 0.4313 - val_loss: 1.0271
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 442s 7s/step - accuracy: 0.4529 - loss: 1.0313 - val_accuracy: 0.4313 - val_loss: 1.0210
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 408s 7s/step - accuracy: 0.4884 - loss: 1.0224 - val_accuracy: 0.4875 - val_loss: 1.0167
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 408s 7s/step - accuracy: 0.4989 - loss: 1.0019 - val_accuracy: 0.4812 - val_loss: 1.0107
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 443s 7s/step - accuracy: 0.4987 - loss: 1.0072 - val_accuracy: 0.5167 - v

In [10]:
# Calculate the average training accuracy across all epochs
average_training_accuracy = sum(training_accuracy) / len(training_accuracy)
print("Average Training Accuracy:", average_training_accuracy)


Average Training Accuracy: 0.4689583361148834


In [11]:
from sklearn.metrics import classification_report
import numpy as np

# Evaluate the model
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Generate classification report
y_true = test_gen.classes
y_pred = np.argmax(model.predict(test_gen), axis=-1)
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=test_gen.class_indices.keys()))


19/19 ━━━━━━━━━━━━━━━━━━━━ 102s 5s/step - accuracy: 0.7142 - loss: 0.8955
Test Accuracy: 53.17%
19/19 ━━━━━━━━━━━━━━━━━━━━ 107s 5s/step
Classification Report:
                                             precision    recall  f1-score   support

                           augmented_images       0.51      0.83      0.63       200
        augmented_images(red vented bulbul)       0.52      0.64      0.57       200
augmented_images(white throated kingfisher)       0.76      0.13      0.22       200

                                   accuracy                           0.53       600
                                  macro avg       0.60      0.53      0.48       600
                               weighted avg       0.60      0.53      0.48       600

